In [1]:
%matplotlib notebook
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hashlib
import sklearn as sk
import os
import time
import datetime
import seaborn as sns

In [60]:
path = '/home/catherinej/Downloads'
datafile = os.path.join(path, 'IrmaMudThicknessComparisons (1).xlsx')
df = pd.read_excel(datafile)

# df = df.set_index(['Site', 'Station'])
df.head()

,Region,Site,Station,Date 1,Date 2,Date 3,Thickness on Date 1 (cm),Thickness on Date 2 (cm),Thickness on Date 3 (cm),Lat,Lon,Notes
0,TTI,CCB,CC1 S1,2017-12-20,2018-01-28,2018-06-20,0,0.0,0,26.022383,-81.734267,NaN
1,TTI,CCB,CC1 S2,2017-12-20,NaT,2018-06-20,0,NaN,0,26.022400,-81.734233,NaN
2,TTI,CCB,CC1 S3,2017-12-20,NaT,2018-06-20,0,NaN,0,26.022300,-81.734267,NaN
3,TTI,CCB,CC1 S4,2017-12-20,NaT,NaT,0,NaN,NaN,26.022317,-81.734183,NaN
4,TTI,CCB,CC2 S1,2017-12-20,2018-01-28,2018-06-20,0,0.0,0,26.022017,-81.734150,NaN


In [61]:
df = (df.replace('<0.1', 0)
        .replace('<1', 0.5)
        .drop('Notes', axis=1)
        )
df.head()

,Region,Site,Station,Date 1,Date 2,Date 3,Thickness on Date 1 (cm),Thickness on Date 2 (cm),Thickness on Date 3 (cm),Lat,Lon
0,TTI,CCB,CC1 S1,2017-12-20,2018-01-28,2018-06-20,0.0,0.0,0.0,26.022383,-81.734267
1,TTI,CCB,CC1 S2,2017-12-20,NaT,2018-06-20,0.0,NaN,0.0,26.022400,-81.734233
2,TTI,CCB,CC1 S3,2017-12-20,NaT,2018-06-20,0.0,NaN,0.0,26.022300,-81.734267
3,TTI,CCB,CC1 S4,2017-12-20,NaT,NaT,0.0,NaN,NaN,26.022317,-81.734183
4,TTI,CCB,CC2 S1,2017-12-20,2018-01-28,2018-06-20,0.0,0.0,0.0,26.022017,-81.734150


In [68]:
df = df[((df['Thickness on Date 1 (cm)'] > df['Thickness on Date 2 (cm)']) & (df['Thickness on Date 2 (cm)'] >  df['Thickness on Date 3 (cm)']))]
hurricane_date = pd.to_datetime('9/10/2017', format='%m/%d/%Y')

diff1 = []
diff2 = []
diff3 = []
for value in df['Date 1']:
    diff = value - hurricane_date
    diff1.append(diff)
df['DiffTime1'] = diff1
df['DiffTime1'] = df['DiffTime1'] / np.timedelta64(1, 'D')

for value in df['Date 2']:
    diff = value - hurricane_date
    diff2.append(diff)
df['DiffTime2'] = diff2
df['DiffTime2'] = df['DiffTime2'] / np.timedelta64(1, 'D')

for value in df['Date 3']:
    diff = value - hurricane_date
    diff3.append(diff)
df['DiffTime3'] = diff3
df['DiffTime3'] = df['DiffTime3'] / np.timedelta64(1, 'D')
df

/home/catherinej/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/catherinej/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/catherinej/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,Region,Site,Station,Date 1,Date 2,Date 3,Thickness on Date 1 (cm),Thickness on Date 2 (cm),Thickness on Date 3 (cm),Lat,Lon,DiffTime1,DiffTime2,DiffTime3
13,TTI,BW2,BW2 S2,2017-12-19,2018-01-28,2018-06-20,11.0,6.5,2.0,25.92488,-81.60933,100.0,140.0,283.0
21,TTI,BW2,BW2 S10,2017-12-19,2018-01-28,2018-06-20,6.0,5.5,4.5,25.92484,-81.60934,100.0,140.0,283.0
37,TTI,UFU,UFU MUD 4,2017-12-18,2018-01-27,2018-06-19,3.0,1.0,0.0,25.90208,-81.51088,99.0,139.0,282.0
38,TTI,UFU,UFU MUD 5,2017-12-18,2018-01-27,2018-06-19,4.0,1.5,0.0,25.90203,-81.51088,99.0,139.0,282.0
55,TTI,FBTBM,FB 2,2017-12-18,2018-01-27,2018-06-19,3.0,1.0,0.0,NaN,NaN,99.0,139.0,282.0
56,TTI,FBTBM,FB 3,2017-12-18,2018-01-27,2018-06-19,3.0,1.0,0.0,NaN,NaN,99.0,139.0,282.0
62,TTI,FBTBM,FB 9,2017-12-18,2018-01-27,2018-06-19,2.0,0.5,0.0,NaN,NaN,99.0,139.0,282.0
63,TTI,FBTBM,FB 10,2017-12-18,2018-01-27,2018-06-19,3.0,2.0,0.0,NaN,NaN,99.0,139.0,282.0
107,LFK,LAD,S5,2017-11-18,2018-03-14,2018-06-15,5.0,1.5,0.0,24.66183,-81.61077,69.0,185.0,278.0
126,LFK,WTZ,S9,2017-11-19,2018-03-14,2018-06-15,0.3,0.2,0.0,24.64770,-81.65384,70.0,185.0,278.0


In [132]:
X = df[['DiffTime1', 'DiffTime2', 'DiffTime3']]
y = df[['Thickness on Date 1 (cm)', 'Thickness on Date 2 (cm)',
       'Thickness on Date 3 (cm)']]


In [133]:
X

,DiffTime1,DiffTime2,DiffTime3
13,100.0,140.0,283.0
21,100.0,140.0,283.0
37,99.0,139.0,282.0
38,99.0,139.0,282.0
55,99.0,139.0,282.0
56,99.0,139.0,282.0
62,99.0,139.0,282.0
63,99.0,139.0,282.0
107,69.0,185.0,278.0
126,70.0,185.0,278.0


In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.2)
print(len(X_3))
print(len(y_3))
X_3_train, X_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, test_size=0.2)

print(len(X_3_train), len(y_3_train))

10
10
8 8


In [135]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lin_reg.intercept_


array([-2756.127 , -1651.3505,  -879.32  ])

In [137]:
print('(Linear Regression) linear model coeff (w):\n{}'
     .format(lin_reg.coef_))
# print('(poly deg 2) linear model intercept (b): {:.3f}'
#      .format(lin_reg.intercept_))
print('(Linear Regression) R-squared score (training): {:.3f}'
     .format(lin_reg.score(X_train, y_train)))
print('(Linear Regression) R-squared score (test): {:.3f}\n'
     .format(lin_reg.score(X_test, y_test)))

(Linear Regression) linear model coeff (w):
[[-4.70000000e+00 -1.93900000e+00  1.23890000e+01]
 [-1.30000000e+00 -2.78500000e-01  6.45350000e+00]
 [-2.57225657e-16  2.60000000e-01  2.99000000e+00]]
(Linear Regression) R-squared score (training): 0.864
(Linear Regression) R-squared score (test): -1.850



In [145]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X_poly, y, random_state=0)

In [146]:
lin_reg.fit(X_train_poly, y_train_poly)
print('(poly deg 2) linear model coeff (w):\n{}'
     .format(lin_reg.coef_))
# print('(poly deg 2) linear model intercept (b): {:.3f}'
#      .format(lin_reg.intercept_))
print('(poly deg 2) R-squared score (training): {:.3f}'
     .format(lin_reg.score(X_train_poly, y_train_poly)))
print('(poly deg 2) R-squared score (test): {:.3f}\n'
     .format(lin_reg.score(X_test_poly, y_test_poly)))

(poly deg 2) linear model coeff (w):
[[ 8.17347225e+06 -8.63176048e+05 -1.31751922e+06 -2.36857562e+06
  -3.29045248e+08 -1.14980557e+08 -1.76460691e+08  1.05483028e+08
  -3.05741904e+08  4.60117184e+08]
 [ 1.63469445e+06 -1.72635210e+05 -2.63503843e+05 -4.73715124e+05
  -6.58090495e+07 -2.29961113e+07 -3.52921381e+07  2.10966055e+07
  -6.11483807e+07  9.20234368e+07]
 [-4.08673612e+06  4.31588024e+05  6.58759608e+05  1.18428781e+06
   1.64522624e+08  5.74902783e+07  8.82303453e+07 -5.27415138e+07
   1.52870952e+08 -2.30058592e+08]]
(poly deg 2) R-squared score (training): 0.849
(poly deg 2) R-squared score (test): -4944733798849208909824.000



In [144]:
PolynomialFeatures?